In [1]:
def make_title(title):
    imsdb_title = ''
    for i in range(len(title)):
        if title[i]=='\n' or title[i]==':':
            continue
        elif title[i] == ' ':
            imsdb_title += '-'
        else:
            imsdb_title += title[i]
    IMSDb_url = 'https://www.imsdb.com/scripts/' + imsdb_title + '.html'

#    tomato_title = ''
#    for i in range(len(title)):
#        if title[i] == '\n':
#            continue
#        elif title[i] == ' ':
#            tomato_title += '_'
#        elif re.match('[A-Z]', title[i]) != None:
#            tomato_title += chr(ord(title[i])+32)
#        else:
#            tomato_title += title[i]
#    tomato_url = 'https://www.rottentomatoes.com/m/' + tomato_title
    
    return IMSDb_url

In [2]:
def pn_score(title=None, name=None):
    import pandas as pd
    import numpy as np
    import os
    import glob
    import pathlib
    import re
    import janome
    import jaconv
    import requests
    from bs4 import BeautifulSoup
    import treetaggerwrapper as ttw
    import warnings
    import matplotlib as mpl
    import matplotlib.pyplot as plt
    
    warnings.simplefilter('ignore')
    
    posi_nega_df = pd.read_csv('dic/pn_en.dic.csv', index_col=0)

    #if title != None:
        #print(title)
    
    if title == None:
        print('title = ', end='')
        title = input()
    if name == None:
        print('hero name = ', end='')
        name = input()
    
    IMSDb_url = make_title(title)
    
    #print(IMSDb_url)
    url = requests.get(IMSDb_url)
    soup = BeautifulSoup(url.content, "html.parser")
    
    # 先頭のBタグの中身(= タイトル)をscriptに格納
    script = soup.find("b")

    # bタグの中身(柱・台詞主の名前)が全部格納されたリスト
    b = soup.findAll("b")
    i = 1

#    title = re.findall('\s*([^\r]*)\r\n', script.text)[0]
#    script = script.next_sibling
#    writer = re.findall('\s*Written\sby\s*([^\r]*)', script.string)[0]

    char_flag = False
    lines = []
    line = ''
    while script.next_sibling != None:
        script = script.next_sibling
        if script.string == b[i].text:
            i += 1
            if re.findall('^\s([^\r])*', script.string) != []:
                if re.findall('\s*([^\s]*)\s*', script.string)[0] == name:
                    char_flag = True
        else:
            s = script.string
            if char_flag:
                j = 0
                while s[0] == ' ':
                    s = s[1:]
                while len(s) > j+3  and  s[j] + s[j+1] + s[j+2] + s[j+3] != '\r\n\r\n':
                    line += s[j].replace('\n', ' ')
                    j += 1
                lines.append(line)
                line = ''
                char_flag = False
    #print('lines len = ', len(lines))
    
    tagger = ttw.TreeTagger(TAGLANG='en')

    word_lists = []
    for i in range(len(lines)):
        for t in tagger.TagText(lines[i]):
            surf = t.split()[0]        # 形態素
            base = t.split()[2]        # 基本形

            # 品詞
            if re.match('V.*', t.split()[1]):
                pos = '動詞'
            elif re.match('WP.*', t.split()[1]) or re.match('PP.*', t.split()[1]):
                pos = '名詞'
            elif re.match('N.*', t.split()[1]):
                pos = '名詞'
            elif re.match('WRB', t.split()[1]) or re.match('EX', t.split()[1]) or re.match('R.*', t.split()[1]):
                pos = '副詞'
            elif re.match('J.*', t.split()[1]) or re.match('PDT', t.split()[1]):
                pos = '形容詞'
            else:
                pos = 'その他'
        
            word_lists.append([i, surf, base, pos])

    word_df = pd.DataFrame(word_lists, columns = ['セリフNo.', '単語', '基本形', '品詞'])
    word_df['品詞'] = word_df['品詞'].apply(lambda x : x.split(',')[0])

    score_result = pd.merge(word_df, posi_nega_df, on = ['基本形', '品詞'], how = 'left')

    result = []
    for i in range(len(score_result['セリフNo.'].unique())):
        temp_df = score_result[score_result['セリフNo.'] == i]
        text = ' '.join(list(temp_df['単語']))
        score = temp_df['スコア'].astype(float).sum()
        score_r = score / temp_df['スコア'].astype(float).count()
        result.append([i, text, score, score_r])

    result_df = pd.DataFrame(result,
                             columns = ['セリフNo.', 'テキスト', '累計スコア', '標準化スコア']
                            ).sort_values(by = 'セリフNo.').reset_index(drop = True)

    result_array = np.nan_to_num(result_df['標準化スコア'].values)

    #fig = plt.figure()
    #ax1 = fig.add_subplot(2, 1, 1)
    #ax2 = fig.add_subplot(2, 1, 2)
    
    #x = np.arange(len(result_array))
    #print('result_array len = ', x)
    #ax1.plot(x, result_array)
    
    return result_array
    
#    n_counter = 0
#    fun = 0
#    pre_n = 0
#    post_p = 0
#    fun_array = []
#    for i in result_array:
#        if i < 0:
#            fun_array.append(0)
#            n_counter += 1
#            pre_n += i
#        elif i > 0:
#            fun = i * n_counter * pre_n * -1
#            fun_array.append(fun)
#            n_counter = 0
#        else:
#            fun_array.append(0)
#    
#    ax2.plot(x, fun_array)
#    
#    return fun_array

In [3]:
with open ('./title/title.txt', 'r', encoding='utf-8') as f:
    titles = f.read().split('\n')
with open('./title/hero_name.txt', 'r', encoding='utf-8') as f:
    names = f.read().split('\n')
with open('./title/score.txt', 'r', encoding='utf-8') as f:
    scores = f.read().split('\n')

In [ ]:
import time
import numpy as np

pn_data = []
used_title = ''
used_score = ''


print('length = ', len(titles)-1)
for i in range(len(titles)):
#for i in range(10):
    print(i, ', ', end='\n')
    if names[i] == '':
        continue  
    data = pn_score(title=titles[i], name=names[i])
    if np.sum(data) == 0:
        continue
    pn_data += [data.tolist()]
    used_title += titles[i] + '\n'
    used_score += scores[i] + '\n' 
    time.sleep(2)


import csv
with open('./data/pn_data.csv', 'w', encoding='utf-8') as file:
    writer = csv.writer(file, lineterminator='\n')
    writer.writerows(pn_data)
with open('./data/title.txt', 'w', encoding='utf-8') as f:
    f.write(used_title)
with open('./data/score.txt', 'w', encoding='utf-8') as f:
    f.write(used_score)


print('Done!')